# Фреймворк PyTorch для разработки искусственных нейронных сетей

## Урок 2. Feed-forward neural network

### Практическое задание

1. Добиться хорошего результата от модели, собранной на занятии (5 угаданныx картинок из 8 предложенных). Варианты изменений:
    - изменение слоёв и их количества;
    - изменение метода оптимизации;
    - изменение процесса обучения;
    - *преобразование данных transform


2. *Переписать данный туториал на PyTorch: https://www.tensorflow.org/tutorials/quickstart/beginner?hl=ru

### Решение. Задание 1

#### Импорт библиотек

In [1]:
import numpy as np

import torch
from torch import nn

import torchvision
from torchvision import transforms

#### Импорт данных

In [2]:
train_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=True,  
                                             transform=transforms.ToTensor(), 
                                             download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

Files already downloaded and verified


In [3]:
test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                            train=False,
                                            download=True,
                                            transform=transforms.ToTensor())

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=8,
                                          shuffle=False)

Files already downloaded and verified


#### Класс модели

In [4]:
class model(nn.Module):
    class base_model(nn.Module):
        def __init__(self, input_dim, output_dim, activation='relu'):
            super().__init__()
            self.layer = nn.Linear(input_dim, output_dim)
            self.activation = activation
            self.dict_activations = {
                'relu': nn.functional.relu,
                'leaky_rely': nn.functional.leaky_relu,
                'sigmoid': torch.sigmoid,
                'softmax': nn.functional.softmax
            }
        
        def forward(self, X_train):
            result = self.layer(X_train)
            try:
                if self.activation == 'softmax':
                    result = self.dict_activations.get(self.activation)(result, dim=-1)
                else:
                    result = self.dict_activations.get(self.activation)(result)
                    
            except TypeError:
                raise Exception(
                    f'Unrecognized activation function name:\n\t{self.activation}' \
                    f'\nPossible names are:\n\t"relu"\n\t"leaky_rely"\n\t"sigmoid"\n\t"softmax"\n'
                )
                
            return result
    
    
    def __init__(self, dict_model_params):
        super().__init__()
        self.dict_model_params = dict_model_params
        
        for layer_name, layer_params in self.dict_model_params.items():
            exec(f'self.layer_{layer_name} = self.base_model(**{layer_params})')
    
    
    def forward(self, X_train):
        X_train = X_train.view(X_train.shape[0], -1)
        
        result = None
        
        local_dict = {'self': self, 'X_train': X_train, 'result': result}
        exec(f'result = self.layer_{[*self.dict_model_params.keys()][0]}(X_train)', globals(), local_dict)
        result = local_dict['result']
        
        for layer_name in [*self.dict_model_params.keys()][1:]:
            exec(f'result = self.layer_{layer_name}(result)', globals(), local_dict)
            result = local_dict['result']
        
        return result
    
    
    def fit(self, X_train, epochs=1):
        self.train()
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(self.parameters(), lr=0.01, momentum=0.0)
        
        for epoch in range(epochs):
            pass
        # In progress.
    
    
    def predict(self, X_test):
        with torch.no_grad():
            self.eval()
            
            result = None

            local_dict = {'self': self, 'X_test': X_test, 'result': result}
            exec(f'result = self.layer_{[*self.dict_model_params.keys()][0]}(X_test)', globals(), local_dict)
            result = local_dict['result']

            for layer_name in [*self.dict_model_params.keys()][1:]:
                exec(f'result = self.layer_{layer_name}(result)', globals(), local_dict)
                result = local_dict['result']

            return result

In [5]:
dict_model_params = {
    '1th': {'input_dim': 1, 'output_dim': 1},
    '2th': {'input_dim': 1, 'output_dim': 1, 'activation': 'sigmoid'}
}

test_model = model(dict_model_params)

test_model

model(
  (layer_1th): base_model(
    (layer): Linear(in_features=1, out_features=1, bias=True)
  )
  (layer_2th): base_model(
    (layer): Linear(in_features=1, out_features=1, bias=True)
  )
)

In [6]:
test_model(torch.FloatTensor([1]))

tensor([[0.3979]], grad_fn=<SigmoidBackward0>)

In [7]:
layer_1th_result = test_model.layer_1th(torch.FloatTensor([1]))

layer_1th_result

tensor([1.1496], grad_fn=<ReluBackward0>)

In [8]:
layer_2th_result =  test_model.layer_2th(layer_1th_result)

layer_2th_result

tensor([0.3979], grad_fn=<SigmoidBackward0>)

#### Построение модели

In [9]:
dict_model_params = {
    '1th': {'input_dim': 3072, 'output_dim': 400, 'activation': 'leaky_rely'},
    '2th': {'input_dim': 400, 'output_dim': 200, 'activation': 'leaky_rely'},
    '3th': {'input_dim': 200, 'output_dim': 100, 'activation': 'leaky_rely'},
    '4th': {'input_dim': 100, 'output_dim': 10, 'activation': 'softmax'},
}

my_model = model(dict_model_params)

my_model

model(
  (layer_1th): base_model(
    (layer): Linear(in_features=3072, out_features=400, bias=True)
  )
  (layer_2th): base_model(
    (layer): Linear(in_features=400, out_features=200, bias=True)
  )
  (layer_3th): base_model(
    (layer): Linear(in_features=200, out_features=100, bias=True)
  )
  (layer_4th): base_model(
    (layer): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [10]:
my_model.train()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(my_model.parameters(), lr=0.01, momentum=0.0)

In [11]:
%%time


num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    running_items = 0.0


    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

        # Обнуляем градиент
        optimizer.zero_grad()
        # Делаем предсказание
        outputs = my_model(inputs)
        # Рассчитываем лосс-функцию
        loss = criterion(outputs, labels)
        # Делаем шаг назад по лоссу
        loss.backward()
        # Делаем шаг нашего оптимайзера
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}')
            running_loss, running_items = 0.0, 0.0

print('Training is finished!')

Epoch [1/5]. Step [1/782]. Loss: 0.036
Epoch [1/5]. Step [301/782]. Loss: 0.036
Epoch [1/5]. Step [601/782]. Loss: 0.036
Epoch [2/5]. Step [1/782]. Loss: 0.036
Epoch [2/5]. Step [301/782]. Loss: 0.036
Epoch [2/5]. Step [601/782]. Loss: 0.036
Epoch [3/5]. Step [1/782]. Loss: 0.036
Epoch [3/5]. Step [301/782]. Loss: 0.036
Epoch [3/5]. Step [601/782]. Loss: 0.036
Epoch [4/5]. Step [1/782]. Loss: 0.036
Epoch [4/5]. Step [301/782]. Loss: 0.036
Epoch [4/5]. Step [601/782]. Loss: 0.036
Epoch [5/5]. Step [1/782]. Loss: 0.036
Epoch [5/5]. Step [301/782]. Loss: 0.036
Epoch [5/5]. Step [601/782]. Loss: 0.036
Training is finished!
Wall time: 38.9 s


Веса не меняются, обучения не происходит.